# Data import

In [2]:
import sqlite3
conn = sqlite3.connect('C:/JupData/mxm_dataset.db')
print(conn)

In [3]:
res = conn.execute("SELECT * FROM sqlite_master WHERE type='table'")
print(res)

Metadata import

In [4]:
conn_tmdb = sqlite3.connect('C:/JupData/track_metadata.db')

In [5]:
res.fetchall()

[('table', 'words', 'words', 2, 'CREATE TABLE words (word TEXT PRIMARY KEY)'),
 ('table',
  'lyrics',
  'lyrics',
  4,
  'CREATE TABLE lyrics (track_id, mxm_tid INT, word TEXT, count INT, is_test INT, FOREIGN KEY(word) REFERENCES words(word))')]

# Playing around in sql

In [ ]:
res = conn.execute("SELECT word FROM words")
len(res.fetchall())

In [ ]:
res = conn.execute("SELECT word FROM words WHERE ROWID BETWEEN 50 AND 60")
res.fetchall()

In [ ]:
res = conn.execute("SELECT track_id FROM lyrics WHERE word='countdown' ORDER BY RANDOM() LIMIT 1")

In [ ]:
res.fetchall()

In [ ]:
res = conn_tmdb.execute("SELECT artist_name, title FROM songs WHERE track_id='TRCKLDK12903CEA5A9'")
res.fetchall()

# Got sick of SQL and switched to pandas

<b> Import data - select quantity with the number after "limit"

In [6]:
#Import words data

import pandas as pd
df = pd.read_sql_query("select * from lyrics limit 5000000;", conn)

In [7]:
print(df.shape)
print(df.tail())
print(len(df['word'].value_counts()))

(5000000, 5)
                   track_id  mxm_tid  word  count  is_test
4999995  TRHRMDQ128F4240653  6643312  gone      4        0
4999996  TRHRMDQ128F4240653  6643312  word      1        0
4999997  TRHRMDQ128F4240653  6643312  hope      4        0
4999998  TRHRMDQ128F4240653  6643312  move      1        0
4999999  TRHRMDQ128F4240653  6643312  name      2        0
5000


In [8]:
#Import track information data to maps track id to title

df_meta=pd.read_sql_query("SELECT * from songs limit 5000000;", conn_tmdb)

In [9]:
print(df_meta.shape)
print(df_meta.tail(1))

(1000000, 14)
                  track_id                                title  \
999999  TRYYYVU12903CD01E3  Fernweh feat. Sektion Kuchikäschtli   

                   song_id     release           artist_id  \
999999  SOWXJXQ12AB0189F43  So Oder So  AR7PLM21187B990D08   

                                 artist_mbid artist_name   duration  \
999999  3af2b07e-c91c-4160-9bda-f0b9e3144ed3       Texta  295.07873   

        artist_familiarity  artist_hotttnesss  year  track_7digitalid  \
999999            0.552977           0.454869  2004           8486723   

        shs_perf  shs_work  
999999        -1         0  


In [15]:
df_titles=df_meta.merge(df, on='track_id', how='right')

In [16]:
#print(df_titles.shape, '\n', df_titles[df_titles['track_id']=='TRAAPKW128F428BC93'])

In [17]:
df_titles.drop(labels=['song_id', 'shs_perf', 'shs_work', 'mxm_tid'],axis=1, inplace=True)

<b> first genres dataset, didn't end up using it

In [10]:
#first genres dataset

df_genres=pd.read_csv('C:\JupData\msd-topMAGD-genreAssignment.cls', delimiter="\t", header=None)

In [11]:
df_genres.columns=['track_id','genre']
print(df_genres.head(1))

             track_id     genre
0  TRAAAAK128F9318786  Pop_Rock


In [12]:
print(df_genres[df_genres['genre']=='Country'])

                  track_id    genre
28      TRAABHC128F933A3F8  Country
93      TRAADXK12903CFAE92  Country
144     TRAAGIU128F931BA5B  Country
148     TRAAGMC128F4292D0F  Country
150     TRAAGNL128F4299BF1  Country
208     TRAAIAE128F42AC53D  Country
323     TRAAMKI128F428537A  Country
326     TRAAMRE128F4298E7D  Country
334     TRAAMXN128F148AD52  Country
343     TRAANEK128F427D2D5  Country
363     TRAANTY12903CAE21E  Country
372     TRAAOEH128F426A82F  Country
374     TRAAOFQ128F92E1E46  Country
391     TRAAORB128F930A97F  Country
394     TRAAORZ128F421CB5C  Country
407     TRAAPKW128F428BC93  Country
415     TRAAPVB12903CDD85D  Country
457     TRAARRM128F42BC439  Country
604     TRAAXTL128F4222D39  Country
624     TRAAYRA128F92E9CC7  Country
720     TRABCLV128F428C59F  Country
726     TRABCSP128F93089E8  Country
758     TRABEDC128F421AFC7  Country
905     TRABKKZ128F427D470  Country
943     TRABLSO128F14A4707  Country
997     TRABNTM128F42A2713  Country
1011    TRABOGP12903CC080D  

<b> second genre dataset, nicer (more merges) 

In [13]:
df_genres2=pd.read_csv('C:\JupData\msd_tagtraum_cd1.cls', delimiter="\t", skiprows=8, usecols=[0,1], header=None)

In [14]:
df_genres2.columns=['track_id', 'genre']
print(df_genres2.head())

             track_id       genre
0  TRAAAAW128F429D538         Rap
1  TRAAABD128F429CF47    Pop_Rock
2  TRAAAED128E0783FAB        Jazz
3  TRAAAEF128F4273421    Pop_Rock
4  TRAAAEM128F93347B9  Electronic


<b> merging genres data with titles and words data

In [18]:
df_merged=df_titles.merge(df_genres2, on='track_id', how='left')

In [19]:
print(df_merged.shape)
print(len(df_merged['word'].unique()))

(5000000, 15)
5000


In [20]:
print(df_merged.head(1))

             track_id             title        release           artist_id  \
0  TRGMMMO128F42AD910  Sear Me MCMXC111  Meisterwork 2  ARW5R811187B98BC96   

                            artist_mbid     artist_name   duration  \
0  2f04ad1f-9f97-41d0-a09c-59c40dfe9cd1  My Dying Bride  441.39057   

   artist_familiarity  artist_hotttnesss  year  track_7digitalid word  count  \
0            0.689401           0.485309     0           1273975    i      1   

   is_test genre  
0        0   NaN  


In [21]:
#checking that we have enough country data, making sure the data looks ok

import numpy as np
df_merged['is_country'] = np.where(df_merged['genre'] ==  'Country', 1,0)

#df_merged['is_country']=0
#df_merged['is_country'][df_merged['genre']=='Country']=1

In [22]:
print(df_merged[df_merged['is_country']==1])

                   track_id              title  \
5304     TRGMRKF128F4275EE7          Pop A Top   
5305     TRGMRKF128F4275EE7          Pop A Top   
5306     TRGMRKF128F4275EE7          Pop A Top   
5307     TRGMRKF128F4275EE7          Pop A Top   
5308     TRGMRKF128F4275EE7          Pop A Top   
5309     TRGMRKF128F4275EE7          Pop A Top   
5310     TRGMRKF128F4275EE7          Pop A Top   
5311     TRGMRKF128F4275EE7          Pop A Top   
5312     TRGMRKF128F4275EE7          Pop A Top   
5313     TRGMRKF128F4275EE7          Pop A Top   
5314     TRGMRKF128F4275EE7          Pop A Top   
5315     TRGMRKF128F4275EE7          Pop A Top   
5316     TRGMRKF128F4275EE7          Pop A Top   
5317     TRGMRKF128F4275EE7          Pop A Top   
5318     TRGMRKF128F4275EE7          Pop A Top   
5319     TRGMRKF128F4275EE7          Pop A Top   
5320     TRGMRKF128F4275EE7          Pop A Top   
5321     TRGMRKF128F4275EE7          Pop A Top   
5322     TRGMRKF128F4275EE7          Pop A Top   


In [23]:
df_model=df_merged[['track_id', 'is_country', 'word', 'count']]

# Sparse matrix representation - efficient

In [24]:
from scipy.sparse import csr_matrix

track_sorted=sorted(df_model.track_id.unique())
words_sorted=sorted(df_model.word.unique())

track_id_u = list(track_sorted)
word_u = list(words_sorted)

data = df_model['count'].tolist()
row = df_model.track_id.astype('category', categories=track_id_u).cat.codes
col = df_model.word.astype('category', categories=word_u).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(track_id_u), len(word_u)))

In [25]:
x_train=np.array(sparse_matrix.todense())

In [26]:
print(x_train.shape)
print(x_train[2860])

(62355, 5000)
[0 0 1 ..., 0 0 0]


In [27]:
y_train=df_model.groupby('track_id', sort=False)['is_country'].agg('mean')
y_train=np.array(y_train)
print(y_train[2980:3000])

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
#CODE TO VALIDATE SPARSE MATRIX ABOVE

#np.set_printoptions(linewidth=100)
#sparse_matrix.todense()
#print(sparse_matrix)
#print(df_model.head(100))

In [28]:
df_model_no_dup=df_model.groupby('track_id')['is_country'].mean()
#print(df_model_no_dup)

#print(len(df_model.loc[df_model.is_country==1]))
y_train2=np.array(df_model_no_dup)

In [29]:
print(y_train2.shape,  y_train2[2980:3000])

(62355,) [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# Naive bayes

In [30]:
# Naive bayes 
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

model1=MultinomialNB()
model1.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [31]:
#Metrics
expected=y_train
predicted=model1.predict(x_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(y_train, predicted)
print(matrix)

training data 
             precision    recall  f1-score   support

          0       0.99      0.94      0.96     61564
          1       0.08      0.45      0.14       791

avg / total       0.98      0.93      0.95     62355

[[57605  3959]
 [  434   357]]


# Extra data pre-processing for more expensive models
SVM, Logistic and RF take a while to run, and could be prone to overfitting. The code below only keeps the top "top" most popular words (change 1 cell down to implement this)

In [32]:
#Creates an array that sums up each column of x_train, sorts it, keeps the top 200 and subsets based on this
x_train_pd=pd.DataFrame(x_train)
x_train_sum=pd.DataFrame(x_train_pd.sum(axis=0))
print(x_train_sum.shape)
x_train_sum['idx']=range(5000)
x_train_sum.sort_values(by=0, axis=0, ascending=False, inplace=True)

(5000, 1)


In [63]:
#Change top below to take more or less words
top=1000

keep_columns=x_train_sum.iloc[top:]['idx']
keep_columns_array=np.array(keep_columns)
x_train_subset=x_train_pd.iloc[:,keep_columns_array]
x_train2=np.array(x_train_subset)

# Logistic

In [56]:
print(x_train_subset)

       1808  4758  3076  1154  2486  4385  2316  1806  3057  78    ...   2137  \
0         0     0     0     0     0     0     0     0     0     0  ...      0   
1         0     3     2     0     1     0     3     0     0     0  ...      0   
2         0     0     0     0     0     6     0     3     0     0  ...      0   
3         0     0     0     0     0     1     0    17     1     0  ...      0   
4         0     0     0     0     0     0     0     0     3     0  ...      0   
5         0     0     0     2     0     0     0     0     0     1  ...      0   
6         0     0     0     0     0     0     0     0     0     0  ...      0   
7         0     0     0     0     0     0     0     0     0     0  ...      0   
8         1     0     0     0     1     0     0     0     0     1  ...      0   
9         1     0     0     1     0     0     0     0     0     0  ...      0   
10        0     0     1     0     0     0     0     1     0     0  ...      0   
11        0     0     0     

In [47]:
print(x_train2.shape, y_train.shape)
print(x_train.shape)
print(x_train, y_train)

(62355, 4000) (62355,)
(62355, 5000)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] [0 0 0 ..., 0 0 0]


In [57]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(class_weight='balanced')

In [66]:
model2=logit.fit(x_train,y_train)

In [ ]:
#Metrics
expected=y_train
predicted=model2.predict(x_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(y_train, predicted)
print(matrix)

# Random forest

In [76]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(class_weight='balanced_subsample')

In [77]:
model3=rfc.fit(x_train, y_train)

In [78]:
#Metrics
expected=y_train
predicted=model3.predict(x_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(y_train, predicted)
print(matrix)

training data 
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     61564
          1       0.99      0.74      0.84       791

avg / total       1.00      1.00      1.00     62355

[[61557     7]
 [  209   582]]


# SVM

In [80]:
from sklearn import svm
svmlin=svm.LinearSVC(class_weight='balanced')

In [81]:
model4=svmlin.fit(x_train, y_train)

In [83]:
#Metrics
expected=y_train
predicted=model4.predict(x_train)
print('training data \n' + metrics.classification_report(expected, predicted))
matrix=metrics.confusion_matrix(y_train, predicted)
print(matrix)

training data 
             precision    recall  f1-score   support

          0       0.99      0.99      0.99     61564
          1       0.49      0.49      0.49       791

avg / total       0.99      0.99      0.99     62355

[[61161   403]
 [  401   390]]


# Close sql connection

In [ ]:
conn.close()
conn_tmdb.close()

# Sparse matrix inefficient
This is legacy code, it's ridiculously slow

In [ ]:
df_model_piv=df_model.pivot(index='track_id', columns='word', values='count')
print(df_model_piv.head(20))

In [ ]:
df_model_piv=df_model_piv.iloc[0:100000]
df_model_piv.replace(to_replace='Nan', value=0, inplace=True)
df_model_piv = df_model_piv.rename(columns={'word': 'track_id'})
print(df_model_piv.head(1))

In [ ]:
df_is_country=df_merged[['track_id', 'is_country']]
df_is_country=df_is_country.groupby(by='track_id', as_index=True).mean()
print(df_is_country[df_is_country.is_country==True])

In [ ]:
df_model_sparse=pd.merge(df_model_piv, df_is_country, left_index=True, right_index=True, how='left')

In [ ]:
#df_model_sparse.to_csv('C:/JupData/abc.csv')

In [ ]:
from sklearn import preprocessing
import numpy as np

y_train=np.array(df_model_sparse['is_country'])
df_model_sparse.drop('is_country', axis=1, inplace=True)

In [ ]:
df_model_sparse.drop('track_id', axis=1, inplace=True)

In [ ]:
x_train=np.array(df_model_sparse)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
print(x_train[2860], y_train[2980:3000])